# ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import xgboost as xgb
import lightgbm as lgb
import shap
from sklearn.metrics import log_loss

# 特徴量エンジニアリング
1. 使わない特徴量を削除
2. カテゴリカルデータをラベルエンコーディング 
3. 欠損値を埋める。(LightGBM)
4. データの特性から特徴量を追加

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
cabin_split = df['Cabin'].str.split('/', expand=True)
# Deck 列を作成
df['Deck'] = cabin_split[0]    
# Side 列を作成し、条件に基づいて数値を割り当て
df['Side'] = cabin_split[2].map({'P': True, 'S': False})
df['Side'] = df['Side'].astype(bool)

# PassengerId から GroupID を抽出
df['GroupID'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
# Group Size を計算
group_sizes = df.groupby('GroupID').size()
df['GroupSize'] = df['GroupID'].apply(lambda x: group_sizes[x])
# Is Alone 特徴量を追加
df['IsAlone'] = (df['GroupSize'] == 1).astype(int)

In [5]:
type(group_sizes)
#type(df.groupby('GroupID'))

pandas.core.series.Series

In [6]:
drop_cols = ['PassengerId', 'Cabin', 'Name','GroupID']
cat_cols = ['HomePlanet', 'CryoSleep', 'Destination','VIP','Deck','Side', 'IsAlone' ,'GroupSize']

In [8]:
df_drop = df.drop(drop_cols, axis=1)

In [9]:
# RoomService の欠損値を条件に基づいて埋める
df_drop['RoomService'] = df_drop.apply(
    lambda row: 0 if pd.isna(row['RoomService']) and row['CryoSleep'] else row['RoomService'],
    axis=1
)


In [10]:
for c in cat_cols:
    le = LabelEncoder()
    le.fit(df_drop[c])
    df_drop[c] = le.transform(df_drop[c])

for c in cat_cols:
    df_drop[c] = df_drop[c].astype('category')

In [11]:
columns_with_nulls = [col for col in df_drop.columns if df_drop[col].isna().any()]
data = df_drop.drop(['Transported'], axis=1)
for c in columns_with_nulls:
    print('-'*100)
    print(c)

# 欠損値がある行とない行を分ける
    train_data = data[data[c].notna()]
    test_data = data[data[c].isna()]

# 訓練データと検証データ
    X_train, X_val, y_train, y_val = train_test_split(
        train_data.drop(c, axis=1), 
        train_data[c],
        test_size=0.2,
        random_state=42
    )
    objective = 'multiclass' if c in ['HomePlanet', 'CryoSleep', 'Destination','VIP','Deck','Side', 'IsAlone' ,'GroupSize'] else 'regression'

# LightGBMのパラメータ
    params = {
        'objective': objective,  # 目的変数が連続値の場合は'regression', カテゴリの場合は'multiclass'
        'learning_rate': 0.1,
        'lambda_l1' : 0.1,
        'num_leaves': 20,
        'random_state': 4
    }

# 訓練データセットを作成
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

# モデルの訓練
    model = lgb.train(params,
                      dtrain,
                      num_boost_round=10000,
                      valid_sets=[dtrain, dval],
                      valid_names=['train', 'valid'],
                      callbacks=[lgb.early_stopping(100),
                      lgb.log_evaluation(500)])

# テストデータで欠損値を予測
    predicted_values = model.predict(test_data.drop(c, axis=1))

# 欠損値の予済結果を元のデータセットに埋める
    data.loc[data[c].isna(), c] = predicted_values

----------------------------------------------------------------------------------------------------
Age
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003760 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1317
[LightGBM] [Info] Number of data points in the train set: 6811, number of used features: 13
[LightGBM] [Info] Start training from score 28.770518
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	train's l2: 150.751	valid's l2: 166.763
----------------------------------------------------------------------------------------------------
RoomService
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002027 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGB

In [12]:
data['total_usage'] = data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa']].sum(axis=1)
data['is_youth'] = np.where(data['Age'] >= 40, 0, 1)

# 訓練データの分割

In [13]:

#X = df_drop.drop(['Transported'], axis=1)
X = data
y = df_drop['Transported']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=0 )
# 最初にデータを訓練データとテストデータに分割します。例えば、全体の20%をテストデータとします。
# 次に、訓練データをさらに訓練データと検証データに分割します。例えば、訓練データのうちさらに25%を検証データとします。
# ここで、訓練データの75%と検証データの25%の割合になりますが、全データの中ではそれぞれ60%と20%になります。
X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=0.20, shuffle=True, stratify=y_train, random_state=0 )

"""
最初にデータを訓練データとテストデータに分割
次に訓練データをさらに訓練データと検証データに分割
X_train, y_train : 訓練データ
X_val, y_val : 検証データ
X_test, y_test : テストデータ
"""

'\n最初にデータを訓練データとテストデータに分割\n次に訓練データをさらに訓練データと検証データに分割\nX_train, y_train : 訓練データ\nX_val, y_val : 検証データ\nX_test, y_test : テストデータ\n'

# モデルの構築

In [15]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val)
lgb_test = lgb.Dataset(X_test, y_test)

params = {
    'objective': 'binary',
    'num_leaves': 10,
    'seed': 0,
    'verbose': -1,
}

In [16]:
model = lgb.train(params,
                      lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_train, lgb_val],
                      valid_names=['train', 'valid'],
                      callbacks=[lgb.early_stopping(100),
                      lgb.log_evaluation(50)])

Training until validation scores don't improve for 100 rounds
[50]	train's binary_logloss: 0.380246	valid's binary_logloss: 0.412331
[100]	train's binary_logloss: 0.34982	valid's binary_logloss: 0.403109
[150]	train's binary_logloss: 0.329363	valid's binary_logloss: 0.40127
[200]	train's binary_logloss: 0.311931	valid's binary_logloss: 0.400651
Early stopping, best iteration is:
[136]	train's binary_logloss: 0.335044	valid's binary_logloss: 0.400159


# テストデータで検証

In [17]:
def predict(submi_data,models):
    output = np.zeros_like(models[0].predict(submi_data))
    for i in range(len(models)):
        output += models[i].predict(submi_data)
       
        
    output = output / len(models)
    return np.round(output).astype(bool)

In [18]:
ac_score = accuracy_score(predict(X_test,[model]), y_test)
pr_score = precision_score(predict(X_test,[model]), y_test)
rc_score = recall_score(predict(X_test,[model]),y_test)
f1 = f1_score(predict(X_test,[model]), y_test)

print('accuracy = %.10f' % (ac_score))
print('precision = %.10f' % (pr_score))
print('recall = %.10f' % (rc_score))
print('F1-score = %.10f' % (f1))

accuracy = 0.8010350776
precision = 0.8310502283
recall = 0.7861771058
F1-score = 0.8079911210
